In [61]:
import gym
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import deque, namedtuple
import random
import numpy as np

Global valriables

In [63]:
GAMMA = 0.99
AVG_OVER = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
env_cart_pole = gym.make('CartPole-v1')
obs, info = env_cart_pole.reset(seed = 0)
#print(f'Observation space: {obs}, Info: {info}')

env_acro_bot = gym.make('Acrobot-v1')
obs, info = env_acro_bot.reset(seed = 0)
#print(f'Observation space: {obs}, Info: {info}')


In [51]:
def print_env_details(env):
    print("Env name: ", str(env.env)[34:-3])
    print(f'Observation space: {env.observation_space.shape[0]}')
    print(f'Action space: {env.action_space.n}')

In [53]:
print_env_details(env_cart_pole)
print_env_details(env_acro_bot)

Env name:  CartPoleEnv<CartPole-v1>
Observation space: 4
Action space: 2
Env name:  AcrobotEnv<Acrobot-v1>
Observation space: 6
Action space: 3


# Dueling-DQN

In [66]:
class QNetwork1(nn.Module):
    def __init__(self, state_size, action_size, seed, fc1_units=128, fc2_units=64, activation = F.relu):
        super(QNetwork1, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.fc_v_1 = nn.Linear(fc2_units, 32)
        self.fc_v_2 = nn.Linear(32, 1)
        self.fc_a_1 = nn.Linear(fc2_units, 32)
        self.fc_a_2 = nn.Linear(32, action_size)
        self.activation = activation

    def forward(self, state):
        x = self.activation(self.fc1(state))
        x = self.activation(self.fc2(x))
        v = self.activation(self.fc_v_1(x))
        v = self.fc_v_2(v)
        a = self.activation(self.fc_a_1(x))
        a = self.fc_a_2(a)
        q = v + (a - a.mean())
        return q

In [65]:
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)
    
    def __len__(self):
        return len(self.memory)
    

In [ ]:
class DDQN_Agent():
    def __init__(self, state_size, action_size, seed):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        


### Type-1 Update rule

\begin{align*}
  Q(s,a;\theta) = V(s;\theta) + (A(s,a;\theta) - \frac{1}{|A|}\sum_{a'\epsilon|A|}A(s,a';\theta))
\end{align*}

### Type-2 Update Rule

\begin{align*}
  Q(s,a;\theta) = V(s;\theta) + (A(s,a;\theta) - \max_{a'\epsilon|A|}\sum_{a'\epsilon|A|}A(s,a';\theta))
\end{align*}

# Monte-Carlo REINFORCE

### W/O Baseline

### W/ Baseline